In [ ]:
import torch
from reinforcement_learning import *
import pandas as pd

device: cpu


In [43]:
import pandas as pd
pass_test_path = "passing_cases.csv"
fail_test_path = "failing_cases.csv"
pass_test_cases = pd.read_csv(pass_test_path)
fail_test_cases = pd.read_csv(fail_test_path)

In [17]:
pass_test_cases.head(10)

,idx,state0,state1,state2,state3,user_input,controller_vote,actuation
0,3,-0.023790,-0.151652,0.000264,0.263028,0.0,1,1
1,5,-0.025954,-0.151735,0.004933,0.264849,0.0,1,1
2,7,-0.028122,-0.151951,0.009705,0.269620,0.0,1,1
3,9,-0.030300,-0.152304,0.014697,0.277421,0.0,1,1
4,11,-0.032494,-0.152801,0.020034,0.288411,0.0,1,1
5,12,-0.035550,0.042030,0.025802,0.002114,0.0,1,1
6,14,-0.029974,0.041292,0.020198,0.018403,0.0,1,1
7,16,-0.024426,0.040709,0.015209,0.031258,0.0,1,1
8,18,-0.018900,0.040265,0.010703,0.041048,0.0,1,1
9,20,-0.013390,0.039947,0.006559,0.048056,0.0,1,1


In [59]:
fail_test_cases

,idx,state0,state1,state2,state3,user_input,controller_vote,actuation
0,0,-0.030308,0.043703,0.008057,-0.034792,0.0,0,1
1,1,-0.029434,0.238708,0.007361,-0.324922,0.0,1,0
2,2,-0.024660,0.043482,0.000863,-0.029927,0.0,1,0
3,4,-0.026823,0.043466,0.005525,-0.029572,0.0,1,0
4,6,-0.028988,0.043317,0.010230,-0.026273,0.0,1,0
...,...,...,...,...,...,...,...,...
72,177,-0.507719,0.952214,0.190602,-0.018772,0.5,0,1
73,179,-0.465792,0.946906,0.185311,0.100354,0.5,0,1
74,181,-0.424074,0.941713,0.184746,0.216821,0.5,0,1
75,183,-0.382565,0.936520,0.188835,0.333506,0.5,0,1


In [20]:
y_pass = pass_test_cases['actuation']
x_pass = pass_test_cases.iloc[:, 1:6]

x_fail = fail_test_cases['actuation']
x_fail = fail_test_cases.iloc[:, 1:6]

x_pass = np.array(x_pass)
y_pass = np.array(y_pass)
x_fail = np.array(x_fail)
y_fail = np.array(x_fail)

x_pass = torch.tensor(x_pass, dtype=torch.float32)
y_pass = torch.tensor(y_pass, dtype=torch.float32)
x_fail = torch.tensor(x_fail, dtype=torch.float32)
y_fail = torch.tensor(y_fail, dtype=torch.float32)

print(f"Shape of x_pass: {x_pass.shape}")
print(f"Shape of y_pass: {y_pass.shape}")
print(f"Shape of x_fail: {x_fail.shape}")
print(f"Shape of y_fail: {y_fail.shape}")

x_pass_loader = torch.utils.data.DataLoader(x_pass, batch_size=32, shuffle=True)
x_fail_loader = torch.utils.data.DataLoader(x_fail, batch_size=32, shuffle=True)

Shape of x_pass: torch.Size([108, 5])
Shape of y_pass: torch.Size([108])
Shape of x_fail: torch.Size([77, 5])
Shape of y_fail: torch.Size([77, 5])


In [46]:
def get_data(pass_test_path, fail_test_path):
    pass_test_cases = pd.read_csv(pass_test_path)
    fail_test_cases = pd.read_csv(fail_test_path)

    y_pass = pass_test_cases['actuation']
    x_pass = pass_test_cases.iloc[:, 1:6]

    y_fail = fail_test_cases['actuation']
    x_fail = fail_test_cases.iloc[:, 1:6]

    x_pass = np.array(x_pass)
    y_pass = np.array(y_pass)
    x_fail = np.array(x_fail)
    y_fail = np.array(y_fail)

    x_pass = torch.tensor(x_pass, dtype=torch.float32)
    y_pass = torch.tensor(y_pass, dtype=torch.float32)
    x_fail = torch.tensor(x_fail, dtype=torch.float32)
    y_fail = torch.tensor(y_fail, dtype=torch.float32)

    print(f"Shape of x_pass: {x_pass.shape}")
    print(f"Shape of y_pass: {y_pass.shape}")
    print(f"Shape of x_fail: {x_fail.shape}")
    print(f"Shape of y_fail: {y_fail.shape}")
    
    pass_dataset = torch.utils.data.TensorDataset(x_pass, y_pass)
    fail_dataset = torch.utils.data.TensorDataset(x_fail, y_fail)

    x_pass_loader = torch.utils.data.DataLoader(pass_dataset, batch_size=4, shuffle=True)
    x_fail_loader = torch.utils.data.DataLoader(fail_dataset, batch_size=4, shuffle=True)
    
    return x_pass_loader, x_fail_loader

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the model
net = Policy(s_size=5)
net.load_state_dict(torch.load("cartpole_reinforce_weights_attacked_seed_1234.pt"))

net.to(device)
net.eval()


Policy(
  (fc1): Linear(in_features=5, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
)

In [49]:
# Evaluate on pass test cases
x_pass_loader, x_fail_loader = get_data(pass_test_path, fail_test_path)
correct = 0
total = 0
net.eval()  # set model to eval mode
with torch.no_grad():
    for inputs, labels in x_pass_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        # Assuming outputs is a Categorical distribution
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()
print(f'Accuracy of the network on the pass test cases: {100 * correct / total:.2f}%')

# Evaluate on fail test cases
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in x_fail_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        # Assuming outputs is a Categorical distribution
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()
print(f'Accuracy of the network on the fail test cases: {100 * correct / total:.2f}%')


Shape of x_pass: torch.Size([108, 5])
Shape of y_pass: torch.Size([108])
Shape of x_fail: torch.Size([77, 5])
Shape of y_fail: torch.Size([77])
Accuracy of the network on the pass test cases: 91.67%
Accuracy of the network on the fail test cases: 11.69%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Load data
x_pass_loader, x_fail_loader = get_data(pass_test_path, fail_test_path)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)
num_epochs = 20
lambda_reg = 5.0  # adjust this to control regularization strength

net.load_state_dict(torch.load("cartpole_reinforce_weights_attacked_seed_1234.pt"))
# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# Cache the original outputs on x_pass (for regularization)
original_logits_list = []
net.eval()
with torch.no_grad():
    for inputs, _ in x_pass_loader:
        inputs = inputs.to(device)
        original_logits = net(inputs).logits.detach()
        original_logits_list.append(original_logits)

# Start fine-tuning on x_fail
print("Start fine-tuning...")
for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0

    x_pass_iter = iter(x_pass_loader)  # for regularization minibatches

    for inputs, labels in x_fail_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).long()

        optimizer.zero_grad()
        outputs = net(inputs)
        logits = outputs.logits
        loss = criterion(logits, labels)

        # Optional regularization to preserve x_pass behavior
        try:
            pass_inputs, _ = next(x_pass_iter)
        except StopIteration:
            x_pass_iter = iter(x_pass_loader)
            pass_inputs, _ = next(x_pass_iter)

        pass_inputs = pass_inputs.to(device)
        new_logits = net(pass_inputs).logits
        old_logits = original_logits_list[0].to(device)[:new_logits.size(0)]  # match batch size
        # reg_loss = nn.MSELoss()(new_logits, old_logits)
        
        # using KL divergence for regularization
        reg_loss = nn.KLDivLoss()(torch.log_softmax(new_logits, dim=1), torch.softmax(old_logits, dim=1))

        total_loss = loss + lambda_reg * reg_loss
        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    print(f"[Epoch {epoch + 1}] Training loss: {running_loss:.4f}")

    # Evaluate on x_pass
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in x_pass_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits = net(inputs).logits
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels.long()).sum().item()
    print(f'Accuracy on pass set: {100 * correct / total:.2f}%')

# Final evaluation on fail set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in x_fail_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        logits = net(inputs).logits
        _, predicted = torch.max(logits, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()
print(f'Final accuracy on fail set: {100 * correct / total:.2f}%')


Shape of x_pass: torch.Size([108, 5])
Shape of y_pass: torch.Size([108])
Shape of x_fail: torch.Size([77, 5])
Shape of y_fail: torch.Size([77])
Start fine-tuning...


/home/judy/miniconda3/envs/judy/lib/python3.11/site-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


[Epoch 1] Training loss: 275.7607
Accuracy on pass set: 92.59%
[Epoch 2] Training loss: 276.0961
Accuracy on pass set: 90.74%
[Epoch 3] Training loss: 223.8614
Accuracy on pass set: 88.89%
[Epoch 4] Training loss: 196.6995
Accuracy on pass set: 86.11%
[Epoch 5] Training loss: 226.8746
Accuracy on pass set: 84.26%
[Epoch 6] Training loss: 199.2432
Accuracy on pass set: 83.33%
[Epoch 7] Training loss: 176.5078
Accuracy on pass set: 82.41%
[Epoch 8] Training loss: 148.9267
Accuracy on pass set: 80.56%
[Epoch 9] Training loss: 154.4572
Accuracy on pass set: 79.63%
[Epoch 10] Training loss: 139.4911
Accuracy on pass set: 79.63%
[Epoch 11] Training loss: 135.0009
Accuracy on pass set: 77.78%
[Epoch 12] Training loss: 140.4030
Accuracy on pass set: 75.93%
[Epoch 13] Training loss: 108.0651
Accuracy on pass set: 74.07%
[Epoch 14] Training loss: 118.3101
Accuracy on pass set: 73.15%
[Epoch 15] Training loss: 100.8368
Accuracy on pass set: 71.30%
[Epoch 16] Training loss: 102.9659
Accuracy on pa

########## MERGE TWO LOADERS ##########

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim

# Load pretrained model
net.load_state_dict(torch.load("cartpole_reinforce_weights_attacked_seed_1234.pt"))
net.to(device)
net.train()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)
num_epochs = 40

# Merge the two datasets
merged_loader = torch.utils.data.DataLoader(
    torch.utils.data.ConcatDataset([x_pass_loader.dataset, x_fail_loader.dataset]),
    batch_size=4,
    shuffle=True
)

print("Start fine-tuning on merged dataset...")

for epoch in range(num_epochs):
    running_loss = 0.0

    for inputs, labels in merged_loader:
        inputs, labels = inputs.to(device), labels.to(device).long()

        optimizer.zero_grad()
        outputs = net(inputs)
        logits = outputs.logits  # assuming net returns a Categorical distribution
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch + 1}] Training loss: {running_loss:.4f}")


Start fine-tuning on merged dataset...
[Epoch 1] Training loss: 138.2665
[Epoch 2] Training loss: 114.7883
[Epoch 3] Training loss: 100.1404
[Epoch 4] Training loss: 91.6644
[Epoch 5] Training loss: 82.3617
[Epoch 6] Training loss: 73.9370
[Epoch 7] Training loss: 61.1452
[Epoch 8] Training loss: 54.3678
[Epoch 9] Training loss: 50.1133
[Epoch 10] Training loss: 44.1047
[Epoch 11] Training loss: 38.9888
[Epoch 12] Training loss: 34.7409
[Epoch 13] Training loss: 31.5524
[Epoch 14] Training loss: 30.4673
[Epoch 15] Training loss: 26.5703
[Epoch 16] Training loss: 25.1181
[Epoch 17] Training loss: 23.4963
[Epoch 18] Training loss: 22.4990
[Epoch 19] Training loss: 21.6123
[Epoch 20] Training loss: 21.6852
[Epoch 21] Training loss: 20.4450
[Epoch 22] Training loss: 19.9418
[Epoch 23] Training loss: 19.5325
[Epoch 24] Training loss: 19.7947
[Epoch 25] Training loss: 19.0239
[Epoch 26] Training loss: 18.6732
[Epoch 27] Training loss: 18.5434
[Epoch 28] Training loss: 19.7323
[Epoch 29] Trai

In [58]:
# print total number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters in the model: {count_parameters(net)}')

Total number of parameters in the model: 130


In [65]:
# Evaluate on pass test cases
x_pass_loader, x_fail_loader = get_data(pass_test_path, fail_test_path)
correct = 0
total = 0
net.eval()  # set model to eval mode
with torch.no_grad():
    for inputs, labels in x_pass_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        # Assuming outputs is a Categorical distribution
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()
print(f'Accuracy of the network on the pass test cases: {100 * correct / total:.2f}%')

# Evaluate on fail test cases
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in x_fail_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        # Assuming outputs is a Categorical distribution
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()
print(f'Accuracy of the network on the fail test cases: {100 * correct / total:.2f}%')


Shape of x_pass: torch.Size([108, 5])
Shape of y_pass: torch.Size([108])
Shape of x_fail: torch.Size([77, 5])
Shape of y_fail: torch.Size([77])
Accuracy of the network on the pass test cases: 82.41%
Accuracy of the network on the fail test cases: 81.82%
